In [1]:
import sys, os
from pathlib import Path
sys.path.append(os.path.abspath('./..'))
import utils

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

datadir = Path('../datadir')

df_session_now = pd.read_excel(datadir / 'session_info.xlsx')
df_trial_now = pd.read_excel(datadir / 'trial_info.xlsx')
df_part_now = pd.read_excel(datadir / 'participant_info.xlsx')

df_part_now.date = pd.to_datetime(df_part_now.date)
df_part_now.date = df_part_now.date.dt.strftime('%Y-%m-%d')


/Users/psr/opt/anaconda3/envs/opencap-nmd/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
sessions = utils.getUserSessions()
sessions_all = []
for s in tqdm(sessions):
    sessions_all.append(utils.getSessionJson(s['id']))
sessions = sessions_all


  0%|          | 0/609 [00:00<?, ?it/s]

In [4]:
part_labels = ['type', 'clinician', 'weight', 'height', 'age', 'sex',
               'gender', 'data_sharing', 'orthotics', 'clock_a', 'brooke',
               'time_10mwt', 'time_10mwrt', 'time_tug_line', 'time_tug_cone',
               'clock_b', 'time_5xsts', 'clock_c', 'time_stairs_up', 'notes_p']


In [5]:
meta = [s for s in sessions]
session_data = []
part_data = []
trial_data = []
for s in tqdm(meta, leave=False):
    sid = s['id']
    pid = s['name']
    session_data.append(dict(
        sid=sid,
        pid=pid,
        created_at=s['created_at'],
        server=s['server'],
        fps=s['meta']['settings']['framerate'] if 'settings' in s['meta'].keys() else np.nan,
        notes_s=np.nan,
    ))
    for t in s['trials']:
        trial = t['name']
        if trial in ('neutral', 'calibration'):
            continue
        trial_data.append(dict(
            sid=sid,
            pid=pid,
            created_at=t['created_at'],
            trial=trial,
            trial_clean=np.nan,
            notes_t=np.nan,
        ))
    # tz = 'America/New_York' if pid.startswith('mdf_') else 'America/Los_Angeles'
    tz = 'America/Los_Angeles'
    part = dict(
        pid=pid,
        date=pd.to_datetime(s['created_at']).tz_convert(tz).strftime('%Y-%m-%d')
    )
    part.update({pl:'' for pl in part_labels})
    part_data.append(part)

df_session = pd.DataFrame(session_data).sort_values('pid')
df_part = pd.DataFrame(part_data).sort_values('pid').drop_duplicates()
df_trial = pd.DataFrame(trial_data).sort_values(['pid', 'trial'])

df_part.shape, df_session.shape, df_trial.shape


  0%|          | 0/609 [00:00<?, ?it/s]

((280, 22), (609, 6), (2714, 6))

In [6]:
df_part_new = df_part_now.set_index(['pid', 'date']).combine_first(df_part.set_index(['pid', 'date']))
cols = list(df_part_now.columns)
cols += [c for c in df_part.columns if c not in cols]
df_part_new = df_part_new.reset_index()[cols]
df_part_new.sort_values(['date', 'pid'], inplace=True)
df_part_new.reset_index( drop=True, inplace=True)
df_part_new


,pid,DIGBI,date,type,location,clinician,weight,height,age,birth_year,...,time_10mwrt,time_tug_line,time_tug_cone,clock_b,time_5xsts,clock_c,time_stairs_up,unable,notes_p,todo
0,58385b9a,NaN,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No REDCap; uses arms on STS,NaN
1,a31757fd,NaN,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No REDCap,NaN
2,3d17b3ee,NaN,2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test subject,NaN
3,6ec44398,NaN,2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No OpenCap trials,NaN
4,7e6bd5de,NaN,2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No OpenCap trials,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,DB-592,NaN,2025-09-06,,NaN,,,,,NaN,...,,,,,,,,NaN,,NaN
818,DB-605,NaN,2025-09-06,,NaN,,,,,NaN,...,,,,,,,,NaN,,NaN
819,DB-619,NaN,2025-09-06,,NaN,,,,,NaN,...,,,,,,,,NaN,,NaN
820,test,NaN,2025-09-06,,NaN,,,,,NaN,...,,,,,,,,NaN,,NaN


In [7]:
df_session_new = df_session_now.set_index(['pid', 'created_at']).combine_first(df_session.set_index(['pid', 'created_at']))
cols = list(df_session_now.columns)
cols += [c for c in df_session.columns if c not in cols]
df_session_new = df_session_new.reset_index()[cols]
df_session_new.sort_values(['created_at', 'pid'], inplace=True)
df_session_new.reset_index( drop=True, inplace=True)
df_session_new


,sid,created_at,pid,url,fps,server,notes_s
0,017ca154-7573-4cd1-ae3f-42dfb8c4f9b0,2023-05-13T21:16:17.904194Z,p012,https://app.opencap.ai/session/017ca154-7573-4...,60.0,171.65.92.206,NaN
1,ee23fbb3-a991-4aa4-9a2f-a213ec9ec6c5,2023-05-13T21:24:25.274931Z,p011,https://app.opencap.ai/session/ee23fbb3-a991-4...,120.0,54.190.39.224,NaN
2,64ac99c8-f9d2-425d-b990-6d770c0288cd,2023-05-13T21:56:04.599875Z,p012,https://app.opencap.ai/session/64ac99c8-f9d2-4...,60.0,171.65.92.206,NaN
3,d6b90c12-92a9-4e5b-9500-54655dde7e63,2023-05-13T21:57:56.146248Z,p012,https://app.opencap.ai/session/d6b90c12-92a9-4...,120.0,171.66.229.49,NaN
4,057d10da-34c7-4fb7-a127-6040010dde06,2023-05-13T22:24:42.055213Z,p013,https://app.opencap.ai/session/057d10da-34c7-4...,60.0,54.190.39.224,NaN
...,...,...,...,...,...,...,...
1027,870331fc-a523-4f0c-9d1c-2ad33459d56a,2025-09-06T22:28:36.063502Z,DB-592,NaN,60,18.236.88.13,NaN
1028,d73e1978-2852-48f5-a62e-c45bb4f8c1ab,2025-09-06T22:34:39.161097Z,DB-587,NaN,60,18.236.88.13,NaN
1029,d09d04aa-d129-4ecf-9010-eed48f47313b,2025-09-06T22:34:43.570182Z,DB-592,NaN,120,171.66.11.144,NaN
1030,e3c6356b-d854-4847-a881-75b0965d5415,2025-09-06T22:39:28.340486Z,DB-619,NaN,60,171.66.11.144,NaN


In [8]:
df_trial_new = df_trial_now.set_index(['sid', 'trial']).combine_first(df_trial.set_index(['sid', 'trial']))
cols = list(df_trial_now.columns)
cols += [c for c in df_trial.columns if c not in cols]
df_trial_new = df_trial_new.reset_index()[cols]
df_trial_new.sort_values(['created_at', 'pid'], inplace=True)
df_trial_new.reset_index( drop=True, inplace=True)
df_trial_new


,pid,created_at,sid,trial,trial_clean,score,notes,bug_status,bug_description,parker/sydney_todo,parker_action,sydney_action,notes_t
0,58385b9a,2022-03-01T22:20:42.799000Z,58385b9a-9eac-44cb-9b4a-40082a442de8,STS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58385b9a,2022-03-01T22:21:11.540000Z,58385b9a-9eac-44cb-9b4a-40082a442de8,TUG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a31757fd,2022-03-01T22:29:58.620000Z,a31757fd-c536-49d5-9c0a-7b33f1c392fc,STS,NaN,NaN,single sts,NaN,NaN,NaN,NaN,NaN,NaN
3,a31757fd,2022-03-01T22:30:29.590000Z,a31757fd-c536-49d5-9c0a-7b33f1c392fc,TUG,NaN,NaN,exits frame,NaN,NaN,NaN,NaN,NaN,NaN
4,9bda23b6,2022-03-02T21:41:32.389000Z,9bda23b6-0b06-4ba2-8141-842461468faf,STS,NaN,NaN,test recording,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7519,DB-619,2025-09-06T22:46:28.892351Z,74ebc946-6c51-457e-a4d7-b2a46338b9f2,10mwt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7520,DB-619,2025-09-06T22:47:07.251429Z,74ebc946-6c51-457e-a4d7-b2a46338b9f2,10mwrt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7521,DB-619,2025-09-06T22:48:03.058538Z,74ebc946-6c51-457e-a4d7-b2a46338b9f2,tug_cone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7522,DB-619,2025-09-06T22:49:08.206330Z,74ebc946-6c51-457e-a4d7-b2a46338b9f2,brooke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_part_new.to_excel(datadir / 'temp/participant_info.xlsx', index=False)
df_session_new.to_excel(datadir / 'temp/session_info.xlsx', index=False)
df_trial_new.to_excel(datadir / 'temp/trial_info.xlsx', index=False)
